# Overview

REFERENCE: https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa


## Setup

In [1]:
#!pip install openai

In [2]:
#!pip install python-dotenv

In [3]:
#!pip install  langchain langchain-experimental

In [4]:
#!pip install psycopg2 python-environ

In [5]:
import os
#import openai
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph

_ = load_dotenv(find_dotenv()) 
#openai.api_key  = os.getenv('OPENAI_API_KEY')
API_KEY = os.getenv('OPENAI_API_KEY')
DBPASS=os.getenv('DBPASS')
DATABASE=os.getenv('DATABASE')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')
NEO4J_URL=os.getenv('NEO4J_URL')
NEO4J_DB=os.getenv('NEO4J_DB')


graph = Neo4jGraph(
    url= NEO4J_URL, username="neo4j", password=NEO4J_PASSWORD
)

In [6]:
print(graph.schema)

Node properties are the following:
Repository {group: STRING, type: STRING, entity: STRING, kind: STRING, name: STRING},API {group: STRING, type: STRING, entity: STRING, kind: STRING, name: STRING, namespace: STRING, onHub: STRING, onManaged: STRING},Users {name: STRING, entity: STRING},Processor {entity: STRING, name: STRING, type: STRING, group: STRING, kind: STRING, namespace: STRING, onHub: STRING, onManaged: STRING, userCreated: BOOLEAN},IntermediateResource {namespace: STRING, entity: STRING, type: STRING, onHub: STRING, onManaged: STRING, name: STRING, group: STRING, kind: STRING, userCreated: BOOLEAN},Gateway {namespace: STRING, entity: STRING, name: STRING, onHub: STRING, onManaged: STRING, type: STRING},CustomResource {name: STRING, entity: STRING, onHub: BOOLEAN, onManaged: BOOLEAN, userCreated: BOOLEAN, namespace: STRING, type: STRING}
Relationship properties are the following:

The relationships are the following:
(:API)-[:Tests]->(:API),(:API)-[:Tests]->(:Repository),(:AP

In [7]:
graph.query(
    """
match (n:Processor{name:'grafana'})-[r:QUERIES|FORWARDS*]-(m) return n ,r,m
"""
)

[{'n': {'namespace': 'open-cluster-management-observability',
   'name': 'grafana',
   'onHub': 'true',
   'type': 'Pod',
   'entity': 'Processor',
   'onManaged': 'false'},
  'r': [({'namespace': 'open-cluster-management-observability',
     'name': 'grafana',
     'onHub': 'true',
     'type': 'Pod',
     'entity': 'Processor',
     'onManaged': 'false'},
    'QUERIES',
    {'namespace': 'open-cluster-management-observability',
     'name': 'rbac-query-proxy',
     'onHub': 'true',
     'type': 'Pod',
     'entity': 'Processor',
     'onManaged': 'false'})],
  'm': {'namespace': 'open-cluster-management-observability',
   'name': 'rbac-query-proxy',
   'onHub': 'true',
   'type': 'Pod',
   'entity': 'Processor',
   'onManaged': 'false'}},
 {'n': {'namespace': 'open-cluster-management-observability',
   'name': 'grafana',
   'onHub': 'true',
   'type': 'Pod',
   'entity': 'Processor',
   'onManaged': 'false'},
  'r': [({'namespace': 'open-cluster-management-observability',
     'name'

In [8]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

### This does not add much value: return_intermediate_steps=True

In [9]:
chaindetail = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
)
result = chaindetail("How does data get into grafana?")
print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Processor)-[:Reads]->(ir:IntermediateResource)-[:Updates]->(cr:CustomResource)
WHERE cr.name = 'grafana'
RETURN p.name, ir.name
Full Context:
[]

> Finished chain.
Intermediate steps: [{'query': "MATCH (p:Processor)-[:Reads]->(ir:IntermediateResource)-[:Updates]->(cr:CustomResource)\nWHERE cr.name = 'grafana'\nRETURN p.name, ir.name"}, {'context': []}]
Final answer: Data can be imported into Grafana through various methods such as connecting to a data source, using plugins or APIs, or by manually uploading data files. Grafana supports a wide range of data sources including databases, cloud services, and monitoring systems. Once the data source is configured, Grafana can query and visualize the data in real-time, allowing users to create interactive dashboards and gain insights from their data.


In [10]:
chain.run("How does data get into grafana?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Processor)-[:Reads]->(ir:IntermediateResource)-[:Updates]->(cr:CustomResource)
WHERE cr.name = 'grafana'
RETURN p.name, ir.name
Full Context:
[]

> Finished chain.


'Data can be imported into Grafana through various methods such as connecting to a data source, using plugins or APIs, or by manually uploading data files. Grafana supports a wide range of data sources including databases, cloud services, and monitoring systems. Once the data source is configured, Grafana can query and visualize the data in real-time, allowing users to create interactive dashboards and gain insights from their data.'

In [11]:
chain.run("What are the names of the things that Users create?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (u:Users)-[:CREATES]->(n)
RETURN n.name
Full Context:
[{'n.name': '{cluster-name}'}, {'n.name': 'AddOnDeploymentConfig-CR'}, {'n.name': '{addon-name}'}, {'n.name': 'KlusterletOperator'}, {'n.name': 'bootstrap-hub-kubeconfig'}, {'n.name': '{cluster-name}'}, {'n.name': 'ThanosObjectStoreSecret'}, {'n.name': 'ObservabilityNamespace'}, {'n.name': 'MCO-CR'}]

> Finished chain.


'The names of the things that Users create are: {cluster-name}, AddOnDeploymentConfig-CR, {addon-name}, KlusterletOperator, bootstrap-hub-kubeconfig, {cluster-name}, ThanosObjectStoreSecret, ObservabilityNamespace, and MCO-CR.'